<a href="https://colab.research.google.com/github/nisanuro/CNG562-Project/blob/master/CNG562_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import re
from pandas import read_csv
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import iplot
from sklearn import model_selection
from sklearn.model_selection import train_test_split, cross_val_score, KFold, learning_curve, StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelEncoder
from tqdm import tqdm
from google.colab import output
from google.colab import drive
import nltk

tqdm.pandas()
nltk.download("wordnet")
drive.mount('/content/drive')
%matplotlib inline

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def compareAccuracy(a, b): 
    print('\nCompare Multiple Classifiers: \n')
    print('K-Fold Cross-Validation Accuracy: \n')
    names = []
    models = []
    resultsAccuracy = []
    models.append(('LR', LogisticRegression()))
    models.append(('LSVM', LinearSVC()))
    models.append(('RF', RandomForestClassifier()))
    for name, model in models:
        model.fit(a, b)
        kfold = model_selection.KFold(n_splits=10, random_state=7)
        accuracy_results = model_selection.cross_val_score(model, a,b, cv=kfold, scoring='accuracy')
        resultsAccuracy.append(accuracy_results)
        names.append(name)
        accuracyMessage = "%s: %f (%f)" % (name, accuracy_results.mean(), accuracy_results.std())
        print(accuracyMessage) 
    # Boxplot
    fig = plt.figure()
    fig.suptitle('Algorithm Comparison: Accuracy')
    ax = fig.add_subplot(111)
    plt.boxplot(resultsAccuracy)
    ax.set_xticklabels(names)
    ax.set_ylabel('Cross-Validation: Accuracy Score')
    plt.show()

In [0]:
lemmatizer = WordNetLemmatizer()
def preprocess(ingredients):
    ingredients_text = ' '.join(ingredients)
    ingredients_text = ingredients_text.lower()
    ingredients_text = ingredients_text.replace('-', '')#wasabe
    ingredients_text = ingredients_text.replace('wasabe', 'wasabi') #for wrong name
    #ingredients_text = ingredients_text.replace('egg whites', 'eggwhites , egg , whites')
    #ingredients_text = ingredients_text.replace('lime juice', 'limejuice')
    #ingredients_text = ingredients_text.replace('clam juice', 'clamjuice')
    #ingredients_text = ingredients_text.replace('lemon juice', 'lemonjuice')
    #ingredients_text = ingredients_text.replace('orange juice', 'orangejuice')
    #ingredients_text = ingredients_text.replace('soy sauce', 'soysauce')
    ingredients_text = ingredients_text.replace('fish sauce', 'fishsauce')
    #ingredients_text = ingredients_text.replace('sesame oil', 'sesameoil')
    #ingredients_text = ingredients_text.replace('olive oil', 'oliveoil')#vegetable oil corn oil
    #ingredients_text = ingredients_text.replace('vegetable oil', 'vegetableoil')
    #ingredients_text = ingredients_text.replace('corn oil', 'cornoil')#rice wine
    #
    ingredients_text = ingredients_text.replace('coconut cream', 'coconutcream')
    #
    ingredients_text = ingredients_text.replace('yellow onion', 'yellowonion')
    ingredients_text = ingredients_text.replace('cream cheese', 'creamcheese') 
    ingredients_text = ingredients_text.replace('baby spinach', 'babyspinach')
    ingredients_text = ingredients_text.replace('coriander seeds', 'corianderseeds')
    ingredients_text = ingredients_text.replace('corn tortillas', 'corntortillas')
    ingredients_text = ingredients_text.replace('rice cakes', 'ricecakes')
    words = []
    for word in ingredients_text.split():
        if re.findall('[0-9]', word): continue
        if len(word) <= 2: continue
        if '’' in word: continue
        word = lemmatizer.lemmatize(word)
        if len(word) > 0: words.append(word)
    return ' '.join(words)

In [0]:
def baseModelComparision(x_train, y_train, test):
    svc_linear = LinearSVC(max_iter=-1)
    svc = SVC(cache_size=1000, max_iter=-1)
    forest = RandomForestClassifier()
    kmeans = KMeans()

    #accuracy_results1 = cross_val_score(svc_linear, x_train, y_train, scoring='accuracy')
    #print("1 {}".format(accuracy_results1))

    accuracy_results4 = cross_val_score(svc, x_train, y_train, scoring='accuracy')
    print("4 {}".format(accuracy_results4))
    
    accuracy_results2 = cross_val_score(forest, x_train, y_train, scoring='accuracy')
    accuracy_results3 = cross_val_score(kmeans, x_train, y_train, scoring='accuracy')

    print("2 {}".format(accuracy_results2))
    print("3 {}".format(accuracy_results3))

In [0]:
def test(x_train, y_train, test):
    estimator = SVC(C=250, # penalty parameter, setting it to a larger value 
	 			 kernel='rbf', # kernel type, rbf working fine here
	 			 degree=3, # default value, not tuned yet
	 			 gamma=1.4, # kernel coefficient, not tuned yet
	 			 coef0=1, # change to 1 from default value of 0.0
	 			 shrinking=True, # using shrinking heuristics
	 			 tol=0.001, # stopping criterion tolerance 
	 			 probability=False, # no need to enable probability estimates
	 			 cache_size=1000, # 200 MB cache size
	 			 class_weight=None, # all classes are treated equally 
	 			 verbose=False, # print the logs 
	 			 max_iter=-1, # no limit, let it run
	 			 decision_function_shape=None, # will use one vs rest explicitly 
	 			 random_state=None)
    classifier = OneVsRestClassifier(estimator)

    scores = cross_validate(classifier, x_train, y_train, cv=3)
    scores['test_score'].mean()

    classifier.fit(x_train, y_train)

    y_pred = label_encoder.inverse_transform(classifier.predict(x_train))
    y_true = label_encoder.inverse_transform(y_train)

    print(f'accuracy score on train data: {accuracy_score(y_true, y_pred)}')

In [0]:
if __name__ == "__main__":
    train_df = pd.read_json('/content/drive/My Drive/CNG562-Project/train.json')
    test_df = pd.read_json('/content/drive/My Drive/CNG562-Project/test.json')
    train=train_df
    test = test_df


In [7]:
    train.head(15)

,id,cuisine,ingredients
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,22213,indian,"[water, vegetable oil, wheat, salt]"
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe..."
5,6602,jamaican,"[plain flour, sugar, butter, eggs, fresh ginge..."
6,42779,spanish,"[olive oil, salt, medium shrimp, pepper, garli..."
7,3735,italian,"[sugar, pistachio nuts, white almond bark, flo..."
8,16903,mexican,"[olive oil, purple onion, fresh pineapple, por..."
9,12734,italian,"[chopped tomatoes, fresh basil, garlic, extra-..."


In [8]:
    total = train.isnull().sum().sort_values(ascending = False)
    percent = (train.isnull().sum()/train.isnull().count()*100).sort_values(ascending = False)
    missing_train_data  = pd.concat([total, percent], axis=1, keys=['Total missing', 'Percent missing'])
    print("             # of Rows, Columns:",train.shape)
    print(missing_train_data.head())

             # of Rows, Columns: (39774, 3)
             Total missing  Percent missing
ingredients              0              0.0
cuisine                  0              0.0
id                       0              0.0


In [0]:
color_theme = dict(color = ['rgba(221,160,221,1)','rgba(169,169,169,1)','rgba(255,160,122,1)','rgba(176,224,230,1)','rgba(169,169,169,1)','rgba(255,160,122,1)','rgba(176,224,230,1)',
                   'rgba(188,143,143,1)','rgba(221,160,221,1)','rgba(169,169,169,1)','rgba(255,160,122,1)','rgba(176,224,230,1)','rgba(189,183,107,1)','rgba(188,143,143,1)','rgba(221,160,221,1)','rgba(169,169,169,1)','rgba(255,160,122,1)','rgba(176,224,230,1)','rgba(169,169,169,1)','rgba(255,160,122,1)'])
temp = train['cuisine'].value_counts()
trace = go.Bar(y=temp.index[::-1],x=(temp)[::-1],orientation = 'h',marker=color_theme)
layout = go.Layout(title = "Count of recipes per cuisine",xaxis=dict(title='Recipe count',tickfont=dict(size=14,)),
                   yaxis=dict(title='Cuisine',titlefont=dict(size=16),tickfont=dict(size=14)),margin=dict(l=200,))
data = [trace]
fig = go.Figure(data=data, layout=layout)
#iplot(fig,filename='basic-bar')

In [10]:
train['x'] = train['ingredients'].progress_apply(lambda ingredients: preprocess(ingredients))
test['x'] = test['ingredients'].progress_apply(lambda ingredients: preprocess(ingredients))
train.head()

100%|██████████| 9944/9944 [00:01<00:00, 8461.45it/s]


,id,cuisine,ingredients,x
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes...",romaine lettuce black olive grape tomato garli...
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g...",plain flour ground pepper salt tomato ground b...
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g...",egg pepper salt mayonaise cooking oil green ch...
3,22213,indian,"[water, vegetable oil, wheat, salt]",water vegetable oil wheat salt
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe...",black pepper shallot cornflour cayenne pepper ...


In [0]:
tfid_vectorizer = make_pipeline(
    TfidfVectorizer(sublinear_tf=True),
    FunctionTransformer(lambda x: x.astype('float'), validate=False)
)

counter_vectorizer = make_pipeline(
    CountVectorizer(),
    FunctionTransformer(lambda x: x.astype('float'), validate=False)
)

hash_vectorizer = make_pipeline(
    HashingVectorizer(),
    FunctionTransformer(lambda x: x.astype('float'), validate=False)
)

In [0]:
x_train_tfid = tfid_vectorizer.fit_transform(train['x'].values)
x_train_tfid.sort_indices()
x_test_tfid = tfid_vectorizer.transform(test['x'].values)

x_train_counter = counter_vectorizer.fit_transform(train['x'].values)
x_train_counter.sort_indices()
x_test_counter = counter_vectorizer.transform(test['x'].values)

x_train_hash = hash_vectorizer.fit_transform(train['x'].values)
x_train_hash.sort_indices()
x_test_hash = hash_vectorizer.transform(test['x'].values)

In [0]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train['cuisine'].values)

In [18]:
baseModelComparision(x_train_tfid, y_train, x_test_tfid)

4 [0.79886864 0.80817096 0.80125707 0.79748586 0.80575811]
2 [0.74758014 0.75675676 0.74644877 0.74494029 0.75496605]
3 [0.02778127 0.03293526 0.08736644 0.01181647 0.01106362]


In [19]:
baseModelComparision(x_train_counter, y_train, x_test_counter)

4 [0.7787555  0.78416091 0.77649277 0.77686989 0.78275082]
2 [0.75901948 0.76555625 0.76065368 0.74934004 0.76489816]
3 [0.02903834 0.03758642 0.07894406 0.11225644 0.04010561]


In [0]:
baseModelComparision(x_train_hash, y_train, x_test_hash)

4 [0.78491515 0.79208045 0.78516656 0.78390949 0.79067136]


In [17]:
test(x_train_tfid, y_train, x_test_tfid)


accuracy score on train data: 0.9996228692110424
